In [ ]:
import re
import os
import nltk
import spacy
import random

from spacy.util import compounding
from spacy.util import minibatch

# Import may not work on Colab (I use jupyter notebook)
from spacy.training.example import Example


from tqdm import tqdm

from nltk.corpus import stopwords

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt


%matplotlib inline


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Evaluation Metric Jaccard Score

def jaccard(str1, str2): 
    """ Returns the Jaccard metric which is the similarty between str1 and str2 """
    
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    
    
    return float(len(c)) / (len(a) + len(b) - len(c))

# Train and Test import


Files

    train.csv - the training set
    test.csv - the test set
    sample_submission.csv - a sample submission file in the correct format

Columns

    textID - unique ID for each piece of text
    text - the text of the tweet
    sentiment - the general sentiment of the tweet
    selected_text - [train only] the text that supports the tweet's sentiment


# Modelling Problem as NER

Named-entity recognition (NER) is a subset of information extraction that acts by locating and classifying named entities mentioned in unstructured text into pre-defined categories.


1. No text pre-processing is used on the training data
2. The spaCy model is employed in a ensemble 3 model configuration trained seperatly for each sentiment
3. Majority voting ia used to make a final prediction of the ensemble.
4. Jaccard metric is used to evaluate the model

In [ ]:
def save_model(output_dir, nlp, output_model_name):
    '''Saves model to a given output directory'''
    
    output_dir = f'{output_dir}'
            
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    nlp.meta["name"] = output_model_name
    nlp.to_disk(output_dir)

    print("Saved model to", output_dir)


In [ ]:
def train_model(train_data, output_dir, n_iter=20, model=None):

    # blank Language class
    nlp = spacy.blank("en")  
    
    # Add pipeline components 
    if "ner" not in nlp.pipe_names:
        
        ner = nlp.create_pipe("ner")
        nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

        
    # go through training data and attach labels
    for _, annotations in train_data:
        
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # disable other pipes during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  

        # if model not set, reinitialize model weights
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()

        # Begin training the model
        for itn in tqdm(range(n_iter)):
            
            # shuffle training data and create minibatches
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            
            
            # Update model using minibatches
            for batch in spacy.util.minibatch(train_data, size=32):
                
                for text, annotations in batch:
                    
                    doc = nlp.make_doc(text)
                    example = Example.from_dict(doc, annotations)
                    nlp.update([example], losses=losses, drop=0.3)
                    print("Losses", losses)

            
    # Save model to output directory
    save_model(output_dir, nlp, 'st_ner')

In [ ]:
def get_model_out_path(sentiment):
    """ Return model saved in DIR for a given sentiment """
    
    model_out_path = None
    
    if sentiment == 'positive':
        
        model_out_path = 'models/model_pos'
        
    elif sentiment == 'negative':
        
        model_out_path = 'models/model_neg'
        
    else:
        model_out_path = 'models/model_neu'
        
    return model_out_path

In [ ]:
def get_training_data(sentiment, data):
    """ Returns a subset of the training data for each sentiment """
    
    train_data = []
    
    # iterate through the dataframe
    for index, row in data.iterrows():
        
        if row.sentiment == sentiment:
            
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
            
    return train_data


In [ ]:
# Read in training and testing set

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Drop NA values

In [ ]:
train = train.dropna()
test = test.dropna()

# Training for positive sentiment

In [ ]:
sentiment = 'positive'

train_data = get_training_data(sentiment, train)
model_path = get_model_out_path(sentiment)

In [ ]:
train_model(train_data, model_path, n_iter=2, model=None)

  0%|          | 0/2 [00:00<?, ?it/s]

Losses {'ner': 15.000000655651093}
Losses {'ner': 23.326146066188812}
Losses {'ner': 36.493389308452606}
Losses {'ner': 43.11711794137955}
Losses {'ner': 49.45837837457657}
Losses {'ner': 53.64777046442032}
Losses {'ner': 68.05975928902626}
Losses {'ner': 84.69796168804169}
Losses {'ner': 88.71275264769793}
Losses {'ner': 100.31677793711424}
Losses {'ner': 108.00167902559042}
Losses {'ner': 117.03318534046412}
Losses {'ner': 119.92843367159367}
Losses {'ner': 122.05409856140614}
Losses {'ner': 123.83349382504821}
Losses {'ner': 126.29441302921623}
Losses {'ner': 128.93145770963747}
Losses {'ner': 129.04532278975967}
Losses {'ner': 129.29987217635062}
Losses {'ner': 131.05514843141282}
Losses {'ner': 132.70018178035116}
Losses {'ner': 132.75809898295392}
Losses {'ner': 134.64171543021564}
Losses {'ner': 136.3762839466388}
Losses {'ner': 138.03159077765773}
Losses {'ner': 139.97108714834596}
Losses {'ner': 139.97136873613653}
Losses {'ner': 141.97202922358372}
Losses {'ner': 143.96808975

Losses {'ner': 539.4751796292664}
Losses {'ner': 541.6040788988137}
Losses {'ner': 543.57322457584}
Losses {'ner': 543.8836906308316}
Losses {'ner': 545.1779856265057}
Losses {'ner': 547.2089284550202}
Losses {'ner': 549.186493897044}
Losses {'ner': 550.8920986886852}
Losses {'ner': 551.6663926505704}
Losses {'ner': 553.5026969191458}
Losses {'ner': 553.752844324459}
Losses {'ner': 555.739122203101}
Losses {'ner': 557.4073573103099}
Losses {'ner': 559.1921279249375}
Losses {'ner': 561.179274337652}
Losses {'ner': 561.1816451459252}
Losses {'ner': 561.2023331192327}
Losses {'ner': 561.3249821042359}
Losses {'ner': 563.2716151728914}
Losses {'ner': 563.4052989165747}
Losses {'ner': 565.3797792370195}
Losses {'ner': 567.6316959992536}
Losses {'ner': 567.633191895275}
Losses {'ner': 569.4460343418904}
Losses {'ner': 571.3947304149218}
Losses {'ner': 573.2125495948463}
Losses {'ner': 575.1843824620088}
Losses {'ner': 577.2218521452766}
Losses {'ner': 578.2345355048247}
Losses {'ner': 580.54

Losses {'ner': 917.0470337672856}
Losses {'ner': 918.0327458781727}
Losses {'ner': 918.0992890394143}
Losses {'ner': 919.2177637032171}
Losses {'ner': 920.7410277286292}
Losses {'ner': 920.8703875701872}
Losses {'ner': 921.1355289605445}
Losses {'ner': 922.2662887431771}
Losses {'ner': 922.3807997803434}
Losses {'ner': 923.4286186408946}
Losses {'ner': 924.2991362262848}
Losses {'ner': 926.3093982933585}
Losses {'ner': 928.8816808626136}
Losses {'ner': 930.0847493151691}
Losses {'ner': 931.9989044859971}
Losses {'ner': 933.8698166275421}
Losses {'ner': 935.6204405884976}
Losses {'ner': 936.6153479822585}
Losses {'ner': 938.5903294946037}
Losses {'ner': 939.0044374996461}
Losses {'ner': 940.6122396871398}
Losses {'ner': 941.4571089159604}
Losses {'ner': 941.4797843671566}
Losses {'ner': 941.6688906841428}
Losses {'ner': 941.7662184432493}
Losses {'ner': 943.2968064964518}
Losses {'ner': 945.1978666545943}
Losses {'ner': 947.6622104806529}
Losses {'ner': 949.5009782231277}
Losses {'ner':

Losses {'ner': 1285.6546722912274}
Losses {'ner': 1287.1538166802304}
Losses {'ner': 1290.535327190572}
Losses {'ner': 1293.1259443564209}
Losses {'ner': 1293.2502890683902}
Losses {'ner': 1295.5190180827196}
Losses {'ner': 1297.143249065077}
Losses {'ner': 1299.1893472883205}
Losses {'ner': 1300.8644156514076}
Losses {'ner': 1302.0295023904487}
Losses {'ner': 1303.002321400681}
Losses {'ner': 1303.6226473119732}
Losses {'ner': 1306.9308613585627}
Losses {'ner': 1309.2394926746638}
Losses {'ner': 1311.2156693438355}
Losses {'ner': 1314.131533602546}
Losses {'ner': 1314.8613819184852}
Losses {'ner': 1317.1307619919244}
Losses {'ner': 1319.1394598706393}
Losses {'ner': 1320.766262603418}
Losses {'ner': 1322.8173976577928}
Losses {'ner': 1325.1249998218518}
Losses {'ner': 1327.151616583822}
Losses {'ner': 1327.4396000285742}
Losses {'ner': 1330.0041451385878}
Losses {'ner': 1331.8538459680822}
Losses {'ner': 1333.5560554833173}
Losses {'ner': 1335.230651556656}
Losses {'ner': 1335.3531808

...

Losses {'ner': 11615.302477663761}
Losses {'ner': 11616.94938420297}
Losses {'ner': 11617.007704680962}
Losses {'ner': 11617.0735015688}
Losses {'ner': 11617.373666620715}
Losses {'ner': 11619.313547518575}
Losses {'ner': 11620.888182700764}
Losses {'ner': 11623.755579491532}
Losses {'ner': 11626.261179112349}
Losses {'ner': 11627.368644480079}
Losses {'ner': 11629.356184924054}
Losses {'ner': 11629.494379876125}
Losses {'ner': 11632.323526619673}
Losses {'ner': 11632.353679263306}
Losses {'ner': 11633.728906397988}
Losses {'ner': 11636.150480732485}
Losses {'ner': 11636.280533406572}
Losses {'ner': 11636.536910628898}
Losses {'ner': 11638.120759348167}
Losses {'ner': 11638.174006373489}
Losses {'ner': 11639.092223865424}
Losses {'ner': 11639.827599625833}
Losses {'ner': 11639.9797710702}
Losses {'ner': 11641.712851960941}
Losses {'ner': 11642.921136644654}
Losses {'ner': 11644.805628920405}
Losses {'ner': 11646.876699860679}
Losses {'ner': 11648.599305906755}
Losses {'ner': 11651.1791

Losses {'ner': 11938.407348248307}
Losses {'ner': 11940.471796659263}
Losses {'ner': 11941.899094327433}
Losses {'ner': 11942.33742953391}
Losses {'ner': 11943.725771634832}
Losses {'ner': 11944.251828486209}
Losses {'ner': 11946.676891150177}
Losses {'ner': 11948.766107429692}
Losses {'ner': 11949.187581057617}
Losses {'ner': 11950.215125413426}
Losses {'ner': 11950.88199920563}
Losses {'ner': 11953.263880929224}
Losses {'ner': 11955.518444273526}
Losses {'ner': 11956.266658327957}
Losses {'ner': 11957.609672598219}
Losses {'ner': 11959.495829107915}
Losses {'ner': 11960.237573812325}
Losses {'ner': 11961.630886251134}
Losses {'ner': 11963.620824417578}
Losses {'ner': 11964.197411273284}
Losses {'ner': 11966.855320030929}
Losses {'ner': 11968.86286292766}
Losses {'ner': 11970.414411973983}
Losses {'ner': 11970.414833061377}
Losses {'ner': 11972.092980470343}
Losses {'ner': 11972.155959502503}
Losses {'ner': 11974.52192439663}
Losses {'ner': 11975.86061337377}
Losses {'ner': 11977.3084

Losses {'ner': 12289.429796612938}
Losses {'ner': 12289.491859184545}
Losses {'ner': 12292.581349106695}
Losses {'ner': 12293.075781285546}
Losses {'ner': 12294.49503717064}
Losses {'ner': 12296.10309530213}
Losses {'ner': 12296.11593959494}
Losses {'ner': 12296.93182700964}
Losses {'ner': 12298.768198210279}
Losses {'ner': 12300.957906951644}
Losses {'ner': 12301.279928852056}
Losses {'ner': 12301.576675793733}
Losses {'ner': 12303.130623403216}
Losses {'ner': 12303.853038748151}
Losses {'ner': 12306.29488590923}
Losses {'ner': 12308.191402286411}
Losses {'ner': 12309.923903151179}
Losses {'ner': 12310.01305228249}
Losses {'ner': 12312.011720875598}
Losses {'ner': 12312.343635854862}
Losses {'ner': 12312.585054770332}
Losses {'ner': 12314.246851826387}
Losses {'ner': 12314.411012109005}
Losses {'ner': 12315.942802177171}
Losses {'ner': 12315.943019816803}
Losses {'ner': 12318.00165573333}
Losses {'ner': 12320.180287673798}
Losses {'ner': 12321.726658519377}
Losses {'ner': 12322.760567

100%|██████████| 2/2 [09:39<00:00, 289.84s/it]

Losses {'ner': 12409.173597323745}
Losses {'ner': 12411.223179088889}
Losses {'ner': 12413.065667516295}
Losses {'ner': 12413.187533806195}
Losses {'ner': 12414.690146141562}
Losses {'ner': 12416.8802292058}
Losses {'ner': 12419.859012984125}
Saved model to models/model_pos


# Training for negative sentiment

In [ ]:
sentiment = 'negative'

train_data = get_training_data(sentiment, train)
model_path = get_model_out_path(sentiment)

In [ ]:
train_model(train_data, model_path, n_iter=2, model=None)

  0%|          | 0/2 [00:00<?, ?it/s]

Losses {'ner': 10.833333790302277}
Losses {'ner': 23.7234069108963}
Losses {'ner': 29.22409874200821}
Losses {'ner': 32.23604643344879}
Losses {'ner': 41.53558528423309}
Losses {'ner': 46.641475543379784}
Losses {'ner': 56.20392124354839}
Losses {'ner': 67.10659189522266}
Losses {'ner': 74.01192639768124}
Losses {'ner': 85.9981113448739}
Losses {'ner': 91.81729763001204}
Losses {'ner': 94.71793366223574}
Losses {'ner': 99.26975129544735}
Losses {'ner': 103.80624825507402}
Losses {'ner': 109.59632997587323}
Losses {'ner': 114.33194263651967}
Losses {'ner': 116.19522264390253}
Losses {'ner': 117.92860692855902}
Losses {'ner': 119.85066289291717}
Losses {'ner': 122.16999852622394}
Losses {'ner': 123.79006786900572}
Losses {'ner': 125.89089908326059}
Losses {'ner': 127.90965511209197}
Losses {'ner': 129.9420718583997}
Losses {'ner': 132.4404863055115}
Losses {'ner': 134.3693952786316}
Losses {'ner': 136.35651480098613}
Losses {'ner': 138.35542809224398}
Losses {'ner': 140.3560896350544}
Lo

Losses {'ner': 531.1468867637321}
Losses {'ner': 533.1667301945703}
Losses {'ner': 535.0337694217063}
Losses {'ner': 536.6728955701653}
Losses {'ner': 538.8426129820552}
Losses {'ner': 546.6378348723854}
Losses {'ner': 548.7019752005237}
Losses {'ner': 551.2511016994624}
Losses {'ner': 553.0985695929678}
Losses {'ner': 553.2161917461168}
Losses {'ner': 555.4844316938949}
Losses {'ner': 557.1974252859187}
Losses {'ner': 559.1318712841136}
Losses {'ner': 561.488510698368}
Losses {'ner': 563.4982057693093}
Losses {'ner': 565.5539968305135}
Losses {'ner': 565.6259908668092}
Losses {'ner': 566.746851547776}
Losses {'ner': 568.7634610628877}
Losses {'ner': 570.83502280193}
Losses {'ner': 572.7728664338339}
Losses {'ner': 574.7132022437892}
Losses {'ner': 576.8147021796835}
Losses {'ner': 576.8783071107974}
Losses {'ner': 579.2543569711245}
Losses {'ner': 581.2426025092939}
Losses {'ner': 583.1381714842212}
Losses {'ner': 585.0871835412911}
Losses {'ner': 587.1105241411553}
Losses {'ner': 588

Losses {'ner': 955.6015567416738}
Losses {'ner': 957.4614469358761}
Losses {'ner': 958.8922269629378}
Losses {'ner': 961.0360587411973}
Losses {'ner': 963.0369206430017}
Losses {'ner': 964.8376118211758}
Losses {'ner': 966.8594871695053}
Losses {'ner': 968.8976192399963}
Losses {'ner': 970.558835085157}
Losses {'ner': 972.3108847393136}
Losses {'ner': 974.3089209048061}
Losses {'ner': 976.2874294155639}
Losses {'ner': 978.7999431337752}
Losses {'ner': 978.9690818288128}
Losses {'ner': 980.8042545511618}
Losses {'ner': 982.8712882131568}
Losses {'ner': 984.7922408002543}
Losses {'ner': 986.5974066133259}
Losses {'ner': 988.0790808022631}
Losses {'ner': 990.2089977877241}
Losses {'ner': 992.0865699123152}
Losses {'ner': 993.1184256728067}
Losses {'ner': 994.940058489554}
Losses {'ner': 996.5025486121359}
Losses {'ner': 996.9893134113944}
Losses {'ner': 999.1128242240947}
Losses {'ner': 1000.7564952494712}
Losses {'ner': 1002.2728117994255}
Losses {'ner': 1003.7925190639784}
Losses {'ner'

Losses {'ner': 1334.214408719332}
Losses {'ner': 1336.3295247199767}
Losses {'ner': 1338.279959632422}
Losses {'ner': 1340.2940545697788}
Losses {'ner': 1342.2919621803908}
Losses {'ner': 1344.1144933594064}
Losses {'ner': 1345.70013719614}
Losses {'ner': 1347.8157605780511}
Losses {'ner': 1349.8072621464275}
Losses {'ner': 1352.6384035542328}
Losses {'ner': 1354.4617577509093}
Losses {'ner': 1356.2990287756152}
Losses {'ner': 1358.1349272475532}
Losses {'ner': 1360.4514268944226}
Losses {'ner': 1360.771820000371}
Losses {'ner': 1362.943696418266}
Losses {'ner': 1364.9318887043282}
Losses {'ner': 1366.8321773819716}
Losses {'ner': 1368.7069218921097}
Losses {'ner': 1368.868969933159}
Losses {'ner': 1368.8733142565325}
Losses {'ner': 1371.1006326301285}
Losses {'ner': 1372.8245545645368}
Losses {'ner': 1374.6042206950187}
Losses {'ner': 1375.7506149830479}
Losses {'ner': 1380.1066531742035}
Losses {'ner': 1382.041072458087}
Losses {'ner': 1382.0744866359155}
Losses {'ner': 1383.29187967

...

Losses {'ner': 11091.744278387667}
Losses {'ner': 11092.016940660962}
Losses {'ner': 11094.383798438586}
Losses {'ner': 11095.504201376982}
Losses {'ner': 11096.623799524094}
Losses {'ner': 11097.97167538087}
Losses {'ner': 11099.468655232626}
Losses {'ner': 11101.802419069709}
Losses {'ner': 11104.585564326071}
Losses {'ner': 11105.096907874646}
Losses {'ner': 11106.43365205608}
Losses {'ner': 11107.139206423883}
Losses {'ner': 11108.225113612367}
Losses {'ner': 11108.79707008495}
Losses {'ner': 11110.477315701657}
Losses {'ner': 11112.740322817986}
Losses {'ner': 11113.7722686692}
Losses {'ner': 11115.33918729804}
Losses {'ner': 11115.438984829721}
Losses {'ner': 11116.068138563329}
Losses {'ner': 11118.017601196541}
Losses {'ner': 11119.62479581875}
Losses {'ner': 11120.232081297328}
Losses {'ner': 11120.232085030295}
Losses {'ner': 11122.510454574278}
Losses {'ner': 11123.978480919048}
Losses {'ner': 11128.231404112974}
Losses {'ner': 11129.010128902324}
Losses {'ner': 11129.735962

Losses {'ner': 11448.691886533546}
Losses {'ner': 11450.520055126943}
Losses {'ner': 11454.305804465552}
Losses {'ner': 11456.318530000912}
Losses {'ner': 11456.745839083536}
Losses {'ner': 11458.91877807989}
Losses {'ner': 11460.622357865013}
Losses {'ner': 11461.333826592669}
Losses {'ner': 11461.425568095896}
Losses {'ner': 11462.991542255917}
Losses {'ner': 11466.425205610454}
Losses {'ner': 11467.609136691888}
Losses {'ner': 11469.616185296707}
Losses {'ner': 11470.959802040637}
Losses {'ner': 11473.833826550566}
Losses {'ner': 11474.88860525227}
Losses {'ner': 11476.344828389227}
Losses {'ner': 11478.67224085696}
Losses {'ner': 11480.759572650473}
Losses {'ner': 11482.306328909042}
Losses {'ner': 11484.127705692577}
Losses {'ner': 11485.83908854831}
Losses {'ner': 11488.268929304537}
Losses {'ner': 11488.28708343641}
Losses {'ner': 11488.670422032346}
Losses {'ner': 11489.963236411306}
Losses {'ner': 11491.556055386565}
Losses {'ner': 11493.497895709095}
Losses {'ner': 11494.2819

Losses {'ner': 11828.292617014462}
Losses {'ner': 11830.08811671703}
Losses {'ner': 11831.324393591522}
Losses {'ner': 11831.516145876287}
Losses {'ner': 11837.0013800575}
Losses {'ner': 11839.88547971475}
Losses {'ner': 11841.88902051737}


100%|██████████| 2/2 [09:02<00:00, 271.03s/it]

Losses {'ner': 11843.912437082708}
Losses {'ner': 11844.983745961174}
Losses {'ner': 11847.371210375446}
Losses {'ner': 11847.905243806834}
Losses {'ner': 11848.051217928069}
Losses {'ner': 11848.785717799252}
Losses {'ner': 11849.551810440222}
Saved model to models/model_neg


# Training for neutral sentiment

In [ ]:
sentiment = 'neutral'

train_data = get_training_data(sentiment, train)
model_path = get_model_out_path(sentiment)

In [ ]:
train_model(train_data, model_path, n_iter=2, model=None)

  0%|          | 0/2 [00:00<?, ?it/s]

Losses {'ner': 17.500000834465027}
Losses {'ner': 31.0131938457489}
Losses {'ner': 42.31914401054382}
Losses {'ner': 50.951987743377686}
Losses {'ner': 58.244224429130554}
Losses {'ner': 68.74318873882294}
Losses {'ner': 76.32570654153824}
Losses {'ner': 85.21795496344566}
Losses {'ner': 89.46685379743576}
Losses {'ner': 93.23049339652061}
Losses {'ner': 98.16641396284103}
Losses {'ner': 103.30101872980595}
Losses {'ner': 105.84369382634759}
Losses {'ner': 107.60623092576861}
Losses {'ner': 110.6940791271627}
Losses {'ner': 113.44563765544444}
Losses {'ner': 114.84856047155336}
Losses {'ner': 116.83201499236748}
Losses {'ner': 118.74915921667707}
Losses {'ner': 120.1908297459886}
Losses {'ner': 122.15250425206614}
Losses {'ner': 123.36041960266084}
Losses {'ner': 125.5580099157487}
Losses {'ner': 127.00134817606522}
Losses {'ner': 129.0632792150227}
Losses {'ner': 130.98744585256827}
Losses {'ner': 132.78836481426333}
Losses {'ner': 134.6338786450051}
Losses {'ner': 136.07197236124375}

Losses {'ner': 622.7484447015065}
Losses {'ner': 622.7484447015065}
Losses {'ner': 624.5891796404052}
Losses {'ner': 628.3525290960897}
Losses {'ner': 628.3525363201692}
Losses {'ner': 628.3525401963077}
Losses {'ner': 628.3525403869825}
Losses {'ner': 628.3525589403808}
Losses {'ner': 628.3539929878789}
Losses {'ner': 628.3541194621879}
Losses {'ner': 628.3543262369896}
Losses {'ner': 628.3559390036089}
Losses {'ner': 628.3565236506978}
Losses {'ner': 628.3565265743639}
Losses {'ner': 628.3565342379724}
Losses {'ner': 628.3565343762698}
Losses {'ner': 628.3565400110637}
Losses {'ner': 628.7452521056923}
Losses {'ner': 628.7452619565055}
Losses {'ner': 628.7452630139779}
Losses {'ner': 632.7448612319828}
Losses {'ner': 632.7448905948165}
Losses {'ner': 634.5840964559081}
Losses {'ner': 634.5840977724255}
Losses {'ner': 634.5840984558705}
Losses {'ner': 634.5842323676022}
Losses {'ner': 634.5847000516881}
Losses {'ner': 634.6049098236124}
Losses {'ner': 634.6049155164523}
Losses {'ner':

Losses {'ner': 728.4250618874324}
Losses {'ner': 728.5954723864091}
Losses {'ner': 730.5902029782088}
Losses {'ner': 734.5901992843283}
Losses {'ner': 734.6485337510029}
Losses {'ner': 734.6485351259468}
Losses {'ner': 734.6485405268271}
Losses {'ner': 734.8737953532147}
Losses {'ner': 734.8737967163681}
Losses {'ner': 736.9329314034273}
Losses {'ner': 736.9329314731895}
Losses {'ner': 736.9329951795928}
Losses {'ner': 736.9330184120985}
Losses {'ner': 738.9212898683911}
Losses {'ner': 739.0946216524693}
Losses {'ner': 739.0946226269}
Losses {'ner': 739.0946226282655}
Losses {'ner': 739.1028640385289}
Losses {'ner': 739.1029732247007}
Losses {'ner': 739.1033953028345}
Losses {'ner': 739.1033956180789}
Losses {'ner': 739.1034840010844}
Losses {'ner': 739.1035907246883}
Losses {'ner': 739.1046839262491}
Losses {'ner': 739.1067648738363}
Losses {'ner': 739.1067649522867}
Losses {'ner': 739.2969319488054}
Losses {'ner': 739.2974116120321}
Losses {'ner': 739.297456701443}
Losses {'ner': 739

Losses {'ner': 805.8763339560957}
Losses {'ner': 805.8763341393063}
Losses {'ner': 809.6102112137024}
Losses {'ner': 809.6102112648796}
Losses {'ner': 809.6103352994729}
Losses {'ner': 809.6103565556942}
Losses {'ner': 809.6103609868161}
Losses {'ner': 809.6130111895199}
Losses {'ner': 809.6130536175665}
Losses {'ner': 813.7148157877556}
Losses {'ner': 813.72960926968}
Losses {'ner': 813.7297747809148}
Losses {'ner': 813.729776340519}
Losses {'ner': 813.737272363765}
Losses {'ner': 813.7415506120343}
Losses {'ner': 813.741603865836}
Losses {'ner': 813.7416068169849}
Losses {'ner': 813.742934596471}
Losses {'ner': 813.7429377808629}
Losses {'ner': 813.7431392947955}
Losses {'ner': 813.7431395356723}
Losses {'ner': 813.7432201813936}
Losses {'ner': 813.7464663990021}
Losses {'ner': 813.7464813767857}
Losses {'ner': 813.7472487099823}
Losses {'ner': 813.7474097560871}
Losses {'ner': 813.753531588762}
Losses {'ner': 813.7535316139503}
Losses {'ner': 813.7535316139806}
Losses {'ner': 813.75

...

Losses {'ner': 3322.7709487855295}
Losses {'ner': 3322.7709487865945}
Losses {'ner': 3324.702815058418}
Losses {'ner': 3324.7029823997154}
Losses {'ner': 3324.702983284533}
Losses {'ner': 3324.7029865513605}
Losses {'ner': 3325.1697419494185}
Losses {'ner': 3325.169741952818}
Losses {'ner': 3325.1697419529814}
Losses {'ner': 3325.169743775794}
Losses {'ner': 3325.169743775794}
Losses {'ner': 3327.169743786718}
Losses {'ner': 3327.1697438026495}
Losses {'ner': 3327.1697747063195}
Losses {'ner': 3327.169775519737}
Losses {'ner': 3327.169777265147}
Losses {'ner': 3327.169777642197}
Losses {'ner': 3327.169827228737}
Losses {'ner': 3327.16984558362}
Losses {'ner': 3327.169848878629}
Losses {'ner': 3327.169848878989}
Losses {'ner': 3327.3396318085715}
Losses {'ner': 3327.3437228137304}
Losses {'ner': 3327.3437228138637}
Losses {'ner': 3327.3437228140133}
Losses {'ner': 3329.343821064262}
Losses {'ner': 3329.3438210643603}
Losses {'ner': 3331.343514002723}
Losses {'ner': 3331.343514002725}
Lo

Losses {'ner': 3410.6247823826693}
Losses {'ner': 3410.63608076787}
Losses {'ner': 3410.636081497818}
Losses {'ner': 3410.636081524323}
Losses {'ner': 3410.636081524323}
Losses {'ner': 3410.6360815437333}
Losses {'ner': 3410.636081716787}
Losses {'ner': 3410.636081717544}
Losses {'ner': 3410.6360817748186}
Losses {'ner': 3410.636081775221}
Losses {'ner': 3410.6360879672147}
Losses {'ner': 3410.636087967216}
Losses {'ner': 3410.636087967297}
Losses {'ner': 3410.636087968407}
Losses {'ner': 3410.6360879688896}
Losses {'ner': 3410.6360879689378}
Losses {'ner': 3412.6043205377414}
Losses {'ner': 3412.604320583107}
Losses {'ner': 3414.5963228323744}
Losses {'ner': 3414.596322832379}
Losses {'ner': 3414.5963228326773}
Losses {'ner': 3417.5765934930005}
Losses {'ner': 3417.6329943706705}
Losses {'ner': 3417.6329943706737}
Losses {'ner': 3417.632999663345}
Losses {'ner': 3417.6329996633513}
Losses {'ner': 3417.632999665722}
Losses {'ner': 3417.633822033973}
Losses {'ner': 3417.633822163708}


100%|██████████| 2/2 [11:57<00:00, 358.64s/it]

Losses {'ner': 3417.633838748008}
Losses {'ner': 3417.6338387706487}
Losses {'ner': 3417.668981246868}


Saved model to models/model_neu


In [ ]:
def predict_entities(text, model):
    
    doc = model(text)
    ent_array = []
    
    # iterate through each named entity
    for ent in doc.ents:

        # create text boundaries
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
            
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    
    return selected_text

In [ ]:
# Next we iterate through the test set and make predictions based on the sentiment of each comment

In [ ]:
model_pos = spacy.load('models/model_pos')
model_neg = spacy.load('models/model_neg')
model_neu = spacy.load('models/model_neu')

jaccard_score = 0

# we iterate through the test set and make predictions based on the sentiment of each comment
for index, row in tqdm(test.iterrows(), total=test.shape[0]):
    
    text = row.text
    
    if row.sentiment == 'neutral':
        jaccard_score += jaccard(predict_entities(text, model_neu), row.text)
        
    elif row.sentiment == 'positive':
        jaccard_score += jaccard(predict_entities(text, model_pos), row.text)
        
    else:
        jaccard_score += jaccard(predict_entities(text, model_neg), row.text) 

print(f'Average Jaccard Score is {jaccard_score / test.shape[0]}') 

100%|██████████| 3534/3534 [00:11<00:00, 297.17it/s]

Average Jaccard Score is 0.8274585726300056


In [ ]:
# Finally we update the test frame with the selected texts in each entry, and display some of the predictions

In [ ]:
selected_texts = []

for index, row in test.iterrows():
    
    text = row.text
    output_str = ""
    
    if row.sentiment == 'neutral' or len(text.split()) < 4:
        selected_texts.append(text)
        
    elif row.sentiment == 'positive':
        selected_texts.append(predict_entities(text, model_pos))
        
    else:
        selected_texts.append(predict_entities(text, model_neg))
        
test['text'] = selected_texts

In [ ]:
display(test.head(10))

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Good,positive
2,eee518ae67,"Recession hit Veronique Branquinho,",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
5,726e501993,that`s great,positive
6,261932614e,I THINK EVERYONE HATES ME ON HERE lol,negative
7,afa11da83f,"soooooo wish i could, but im in school and my...",negative
8,e64208b4ef,and within a short time of the last clue all ...,neutral
9,37bcad24ca,What did you get? My day is alright.. haven`...,neutral
